In [11]:
library(xts);library(caret);library(dplyr);library(zoo)
library(tidyverse);library(lubridate);library(data.table)
library(ggplot2);library(timeDate);library(Metrics);
library(hydroGOF);library(imputeTS);library(readxl);library(forecastML)

In [4]:
#df <- read.csv("2016_20_Anand_Vihar.csv", na.strings = "None")
## Data cleaning 
## Reading the excel file, starting with 12 line and giving NA as None
df <- read_xlsx('Data/Anand_Vihar_16_21.xlsx',skip = 12,na = "None")
## converting date into proper date format and selecting the relevant variable needed
df <- df %>% mutate(date = dmy_hm(`From Date`,tz="Asia/Kolkata"),PM25=`PM2.5`)%>%
select(date,PM25,AT,RH,WS,SR,WD)
head(df)

date,PM25,AT,RH,WS,SR,WD
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2016-01-01 01:00:00,376.00,11.43,69.00,0.30,12.25,216.25
2016-01-01 02:00:00,480.50,11.28,71.50,0.35,12.58,230.92
2016-01-01 03:00:00,486.67,11.71,71.83,0.31,12.33,203.25
2016-01-01 04:00:00,441.17,11.13,73.08,0.30,12.75,126.92
2016-01-01 05:00:00,594.83,11.01,74.75,0.30,12.58,196.42
2016-01-01 06:00:00,441.33,11.49,74.17,0.37,13.83,80.58


In [5]:
## Converting the data into daily time series
daily_df <- df %>%
  select(date, PM25) %>% # to check the model for PM2.5 replace PM2.5 by PM2.5
  # extract date features
  mutate(
    date = as.POSIXct(date, format = "%d-%m-%Y %H:%M"), #Date format
    hour = hour(date),
    day = day(date),
    month = month(date),
    year = year(date),
    dates = as.Date(paste0(day, "-", month, "-", year), "%d-%m-%Y"),
    PM2.5 = ifelse(PM25 <=10 ,NA,PM25)
  ) %>%
  # from hourly to daily
  group_by(dates, day, month, year) %>% ## Convert hourly to daily 
  summarise(PM2.5_daily = mean(PM2.5, na.rm = TRUE)) %>%
  mutate(week = week(dates))

## Putting NA if there are null values 
daily_df$PM2.5_daily[is.nan(daily_df$PM2.5_daily)] <- NA


`summarise()` has grouped output by 'dates', 'day', 'month'. You can override using the `.groups` argument.



In [78]:
######################## Seasonal Indexes #####################################

## We creating index of seasons from the data till May 2019

seasonalIndex <- daily_df %>% filter(dates < as.Date("2020-12-31"))


############################### daily seasonality  #######################
daily_s <- seasonalIndex %>% group_by(year, month, day) %>%
  summarize(average = mean(PM2.5_daily, na.rm = TRUE))

daily_s$average[is.nan(daily_s$average)] <- NA

## Spanning the year wise value column wise
daily_s <- daily_s %>% ungroup() %>%
  spread(year, average)
head(daily_s)

# Calcuating the mean daily index over the years (Except first and second columns)
daily_s$meanindex <- rowMeans(daily_s[, -c(1,2)], na.rm = TRUE)

## Summary mean is the mean of Mean index
summary_mean <- daily_s %>% ungroup() %>%
  summarize(summary_mean = mean(meanindex, na.rm = T))

## We merge the Summary mean with our old data frame 
daily_s <- merge(daily_s, summary_mean)

## Finally calculating the daily index by mean index/ summary mean * 100
daily_s <-
  daily_s %>% mutate(dailyindex = meanindex / summary_mean * 100)

head(daily_s)


`summarise()` has grouped output by 'year', 'month'. You can override using the `.groups` argument.



month,day,2016,2017,2018,2019,2020
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,359.5730,199.8817,481.6250,380.2292,367.3750
1,2,327.9150,257.1204,359.6667,418.4688,361.6875
1,3,341.3188,271.6446,287.7083,412.8854,378.6765
1,4,480.0688,270.1217,351.6250,291.9479,216.5625
1,5,433.3537,246.8504,306.2500,241.6667,190.5938
1,6,371.7995,189.3525,324.7500,191.8542,166.8646


,month,day,2016,2017,2018,2019,2020,meanindex,summary_mean,dailyindex
,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1,359.5730,199.8817,481.6250,380.2292,367.3750,357.7368,139.2887,256.8312
2,1,2,327.9150,257.1204,359.6667,418.4688,361.6875,344.9717,139.2887,247.6667
3,1,3,341.3188,271.6446,287.7083,412.8854,378.6765,338.4467,139.2887,242.9822
4,1,4,480.0688,270.1217,351.6250,291.9479,216.5625,322.0652,139.2887,231.2214
5,1,5,433.3537,246.8504,306.2500,241.6667,190.5938,283.7429,139.2887,203.7085
6,1,6,371.7995,189.3525,324.7500,191.8542,166.8646,248.9242,139.2887,178.7110


In [7]:
############################### weekly seasonality ###########
weekly_s <- seasonalIndex %>%
  group_by(year, week) %>%
  summarize(average = mean(PM2.5_daily, na.rm = TRUE))

## NA if the average in null
weekly_s$average[is.nan(weekly_s$average)] <- NA

## Spreading across weeks
weekly_s <- weekly_s %>% ungroup() %>%
  spread(year, average)

head(weekly_s)

# Calcuating the mean weekly index over the years
weekly_s$meanindex <- rowMeans(weekly_s[, -1], na.rm = TRUE)

## Calculting summary means
summary_mean <- weekly_s %>% ungroup() %>%
  summarize(summary_mean = mean(meanindex, na.rm = T))

## Adding summary mean
weekly_s <- merge(weekly_s, summary_mean)

## Calculating weekly index
weekly_s <-
  weekly_s %>% mutate(weeklyindex = meanindex / summary_mean * 100)

head(weekly_s)

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.



week,2016,2017,2018,2019,2020
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,385.6715,239.1619,351.9375,322.8420,280.2933
2,349.6154,169.9775,265.8299,259.8899,175.8200
3,250.1298,165.8277,298.4800,283.6310,148.2443
4,324.5103,166.7606,201.9841,118.4670,190.7699
5,302.3383,178.0511,169.1964,189.1205,110.1656
6,219.2755,151.1387,192.8178,166.1840,158.7121


,week,2016,2017,2018,2019,2020,meanindex,summary_mean,weeklyindex
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,385.6715,239.1619,351.9375,322.8420,280.2933,315.9812,143.0075,220.9543
2,2,349.6154,169.9775,265.8299,259.8899,175.8200,244.2265,143.0075,170.7788
3,3,250.1298,165.8277,298.4800,283.6310,148.2443,229.2626,143.0075,160.3150
4,4,324.5103,166.7606,201.9841,118.4670,190.7699,200.4984,143.0075,140.2013
5,5,302.3383,178.0511,169.1964,189.1205,110.1656,189.7744,143.0075,132.7024
6,6,219.2755,151.1387,192.8178,166.1840,158.7121,177.6256,143.0075,124.2072


In [8]:
############################### monthly seasonality ##############################

## Grouping by year and month
monthly_s <- seasonalIndex %>%
  group_by(year, month) %>%
  summarize(average = mean(PM2.5_daily, na.rm = TRUE))

## Adding NA values if there are any null values of any sort 
monthly_s$average[is.nan(monthly_s$average)] <- NA

## Spreading across months 
monthly_s <- monthly_s %>% ungroup() %>%
  spread(year, average)

head(monthly_s)

# Calcuating the mean weekly index over the years (-1 to remove first column which is month)
monthly_s$meanindex <- rowMeans(monthly_s[, -1], na.rm = TRUE)

## Summary mean of the mean index
summary_mean <- monthly_s %>% ungroup() %>%
  summarize(summary_mean = mean(meanindex, na.rm = T))

## Merging the summary mean with the data frame
monthly_s <- merge(monthly_s, summary_mean)

## Adding the Monthly index
monthly_s <-
  monthly_s %>% mutate(monthlyindex = meanindex / summary_mean * 100)
head(monthly_s)

`summarise()` has grouped output by 'year'. You can override using the `.groups` argument.



month,2016,2017,2018,2019,2020
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,329.30722,182.5829,265.5462,235.87494,182.17713
2,166.68875,138.3392,161.2071,141.88165,123.95736
3,121.33565,113.1674,119.2262,106.87930,67.84059
4,149.94428,129.5753,115.1012,98.01637,47.16792
5,101.07827,150.6213,107.3090,111.00798,73.74467
6,92.98106,102.3051,112.9941,73.47000,64.44216


,month,2016,2017,2018,2019,2020,meanindex,summary_mean,monthlyindex
,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,329.30722,182.5829,265.5462,235.87494,182.17713,239.09769,141.2325,169.29371
2,2,166.68875,138.3392,161.2071,141.88165,123.95736,146.41480,141.2325,103.66937
3,3,121.33565,113.1674,119.2262,106.87930,67.84059,105.68983,141.2325,74.83395
4,4,149.94428,129.5753,115.1012,98.01637,47.16792,107.96101,141.2325,76.44206
5,5,101.07827,150.6213,107.3090,111.00798,73.74467,108.75225,141.2325,77.00230
6,6,92.98106,102.3051,112.9941,73.47000,64.44216,89.23849,141.2325,63.18553


In [9]:

############# Join the indexes to original daily data
daily_s <- daily_s %>% select(month, day, dailyindex)
daily_df <- daily_df %>% left_join(daily_s, by = c("month", "day"))

monthly_s <- monthly_s %>% select(month, monthlyindex)
daily_df <- daily_df %>% left_join(monthly_s, by = c("month"))

weekly_s <- weekly_s %>% select(week, weeklyindex)
daily_df <- daily_df %>% left_join(weekly_s, by = c("week"))

head(daily_df)
## Removing the unnecessary data frames 
rm(daily_s)
rm(monthly_s)
rm(weekly_s)
rm(summary_mean)
rm(seasonalIndex)


dates,day,month,year,PM2.5_daily,week,dailyindex,monthlyindex,weeklyindex
<date>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
2016-01-01,1,1,2016,359.5730,1,256.8312,169.2937,220.9543
2016-01-02,2,1,2016,327.9150,1,247.6667,169.2937,220.9543
2016-01-03,3,1,2016,341.3188,1,242.9822,169.2937,220.9543
2016-01-04,4,1,2016,480.0688,1,231.2214,169.2937,220.9543
2016-01-05,5,1,2016,433.3537,1,203.7085,169.2937,220.9543
2016-01-06,6,1,2016,371.7995,1,178.7110,169.2937,220.9543


In [57]:
################### Data manipulation using forecastML
## Fill_gaps is a function from forecatML to see if there are missing value it creates
## an evenly spaced data frame
daily_ts <- fill_gaps(daily_df, date_col = 1, frequency = "1 day")
dates <- daily_ts$dates
daily_ts$dates <- NULL
daily_ts$day <- NULL
daily_ts$month <- NULL
daily_ts$week <- NULL
head(daily_ts)


,year,PM2.5_daily,dailyindex,monthlyindex,weeklyindex
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,2016,359.5730,256.8312,169.2937,220.9543
2,2016,327.9150,247.6667,169.2937,220.9543
3,2016,341.3188,242.9822,169.2937,220.9543
4,2016,480.0688,231.2214,169.2937,220.9543
5,2016,433.3537,203.7085,169.2937,220.9543
6,2016,371.7995,178.7110,169.2937,220.9543


In [68]:
## Now creating a training data frame with lookback of 7 days 
data_lagged <-
  create_lagged_df(
    daily_ts, ## data frame
    type = "train", ## training 
    outcome_col = 2,## Output column is PM2.5 
    horizons = 1, ## Horizons is used if you want to use different models in different time frames
    lookback = c(1:15),## Lookback of 7 hour
    dates = dates, ## dates vector
    dynamic_features = c("year", "dailyindex", "monthlyindex", "weeklyindex"), ## dynamic features of index
    frequency = "1 day" ## Frequency is 1 day
  )
## Looking at the first horizon data frame
head(data_lagged$horizon_1)


,PM2.5_daily,PM2.5_daily_lag_1,PM2.5_daily_lag_2,PM2.5_daily_lag_3,PM2.5_daily_lag_4,PM2.5_daily_lag_5,PM2.5_daily_lag_6,PM2.5_daily_lag_7,PM2.5_daily_lag_8,PM2.5_daily_lag_9,PM2.5_daily_lag_10,PM2.5_daily_lag_11,PM2.5_daily_lag_12,PM2.5_daily_lag_13,PM2.5_daily_lag_14,PM2.5_daily_lag_15,year,dailyindex,monthlyindex,weeklyindex
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
16,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,433.3537,480.0688,341.3188,327.9150,359.5730,2016,140.0407,169.2937,160.3150
17,200.1717,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,433.3537,480.0688,341.3188,327.9150,2016,164.4781,169.2937,160.3150
18,335.4121,200.1717,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,433.3537,480.0688,341.3188,2016,201.6707,169.2937,160.3150
19,322.0700,335.4121,200.1717,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,433.3537,480.0688,2016,195.6573,169.2937,160.3150
20,307.6042,322.0700,335.4121,200.1717,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,433.3537,2016,178.2341,169.2937,160.3150
21,234.4342,307.6042,322.0700,335.4121,200.1717,143.2367,182.7496,259.6646,313.1204,286.4671,409.0429,281.5725,169.4908,519.5942,468.0200,371.7995,2016,150.1126,169.2937,140.2013


In [69]:
data_lagged <- data_lagged$horizon_1
## Filtering in train and test and then removing the NA values
#data_train <- data_lagged %>% filter(dates <  ymd("2021-01-01")) %>% drop_na()
#data_test <- data_lagged %>% filter(dates >= ymd("2021-01-01")) %>% drop_na()
#head(data_test)
data_train <- data_lagged %>% filter(year < 2021) %>% drop_na()
data_test <- data_lagged %>% filter(year >= 2021) %>% drop_na()
head(data_test)

,PM2.5_daily,PM2.5_daily_lag_1,PM2.5_daily_lag_2,PM2.5_daily_lag_3,PM2.5_daily_lag_4,PM2.5_daily_lag_5,PM2.5_daily_lag_6,PM2.5_daily_lag_7,PM2.5_daily_lag_8,PM2.5_daily_lag_9,PM2.5_daily_lag_10,PM2.5_daily_lag_11,PM2.5_daily_lag_12,PM2.5_daily_lag_13,PM2.5_daily_lag_14,PM2.5_daily_lag_15,year,dailyindex,monthlyindex,weeklyindex
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,477.9559,298.4167,191.1667,141.8438,95.98958,269.50000,274.85417,230.68750,357.85417,421.92105,470.5625,274.9271,224.0833,167.9688,150.4792,156.1364,2021,256.8312,169.2937,220.9543
2,317.2344,477.9559,298.4167,191.1667,141.84375,95.98958,269.50000,274.85417,230.68750,357.85417,421.9211,470.5625,274.9271,224.0833,167.9688,150.4792,2021,247.6667,169.2937,220.9543
3,148.0729,317.2344,477.9559,298.4167,191.16667,141.84375,95.98958,269.50000,274.85417,230.68750,357.8542,421.9211,470.5625,274.9271,224.0833,167.9688,2021,242.9822,169.2937,220.9543
4,112.4271,148.0729,317.2344,477.9559,298.41667,191.16667,141.84375,95.98958,269.50000,274.85417,230.6875,357.8542,421.9211,470.5625,274.9271,224.0833,2021,231.2214,169.2937,220.9543
5,85.0000,112.4271,148.0729,317.2344,477.95588,298.41667,191.16667,141.84375,95.98958,269.50000,274.8542,230.6875,357.8542,421.9211,470.5625,274.9271,2021,203.7085,169.2937,220.9543
6,148.0000,85.0000,112.4271,148.0729,317.23438,477.95588,298.41667,191.16667,141.84375,95.98958,269.5000,274.8542,230.6875,357.8542,421.9211,470.5625,2021,178.7110,169.2937,220.9543


In [70]:
#data_train$year <- as.factor(data_train$year)
# Defining train control for 10 fold Crossvalidation 
tc <- trainControl(method="cv", number = 10)
lm1_cv <- train(PM2.5_daily ~., method="lm",
                   data = data_train,trControl=tc)
lm1_cv

Linear Regression 

1186 samples
  19 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1067, 1066, 1067, 1068, 1067, 1067, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  52.01668  0.8184375  35.92936

Tuning parameter 'intercept' was held constant at a value of TRUE

In [71]:
lasso1_cv <- train(PM2.5_daily ~., method="glmnet",data = data_train,trControl=tc,
                      tuneGrid=expand.grid(alpha=1,lambda=0))
lasso1_cv

glmnet 

1186 samples
  19 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1066, 1066, 1066, 1069, 1067, 1070, ... 
Resampling results:

  RMSE      Rsquared   MAE     
  51.99433  0.8186676  35.89352

Tuning parameter 'alpha' was held constant at a value of 1
Tuning
 parameter 'lambda' was held constant at a value of 0

In [72]:
rf1_cv <- train(PM2.5_daily ~., method="rf",
                   data = data_train,trControl=tc)
rf1_cv

Random Forest 

1186 samples
  19 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1067, 1068, 1068, 1066, 1068, 1068, ... 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
   2    59.11863  0.7686363  39.80967
  10    55.77452  0.7903816  36.54191
  19    56.28936  0.7859226  36.71801

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 10.

In [73]:
tg_gbm_d <- expand.grid(shrinkage = seq(0.1, 1, by = 0.2), 
                      interaction.depth = c(1, 3, 7, 10),
                      n.minobsinnode = c(2, 5, 10),
                      n.trees = c(100, 300, 500))
# Gradient Boosting method
gbm1_cv <- caret::train(PM2.5_daily ~., 
                             method="gbm",tuneGrid=tg_gbm_d,
                           data = data_train,trControl=tc,verbose=FALSE)
gbm1_cv

Stochastic Gradient Boosting 

1186 samples
  19 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1067, 1066, 1068, 1067, 1069, 1067, ... 
Resampling results across tuning parameters:

  shrinkage  interaction.depth  n.minobsinnode  n.trees  RMSE       Rsquared 
  0.1         1                  2              100       56.20133  0.7872956
  0.1         1                  2              300       56.57186  0.7837922
  0.1         1                  2              500       57.23994  0.7786146
  0.1         1                  5              100       56.47773  0.7844769
  0.1         1                  5              300       56.70785  0.7816904
  0.1         1                  5              500       57.19090  0.7774499
  0.1         1                 10              100       56.51974  0.7840431
  0.1         1                 10              300       56.84412  0.7800067
  0.1         1                 10              500       57.23845  0

In [74]:
nnetGrid <-  expand.grid(size = seq(from = 5, to = 40, by = 10),
                         decay = seq(from = 0.1, to = 0.5, by = 0.1))
set.seed(123)
# Neural Network model
nn1_cv <- caret::train(PM2.5_daily ~., method="nnet",
                          data = data_train,tuneGrid=nnetGrid,trControl=tc,linout=TRUE, linear.output=TRUE,trace = FALSE,metric="RMSE",verbose=FALSE)
nn1_cv

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Neural Network 

1186 samples
  19 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 1067, 1066, 1069, 1067, 1068, 1069, ... 
Resampling results across tuning parameters:

  size  decay  RMSE      Rsquared   MAE     
   5    0.1    92.09726  0.5787070  68.49808
   5    0.2    80.08531  0.5806312  56.79226
   5    0.3    84.42034  0.5766972  61.34240
   5    0.4    82.42574  0.6050630  58.64293
   5    0.5    82.47228  0.5416981  58.66775
  15    0.1    99.29400  0.4529926  75.02134
  15    0.2    81.30647  0.5596245  59.32310
  15    0.3    81.78903  0.5518439  58.49250
  15    0.4    78.60378  0.5854918  56.01394
  15    0.5    80.38876  0.5656422  55.70822
  25    0.1    93.00340  0.5725823  69.53990
  25    0.2    85.75453  0.5444121  62.20521
  25    0.3    78.36708  0.5942091  56.22462
  25    0.4    77.75022  0.6056934  55.42235
  25    0.5    81.28960  0.5547292  58.07024
  35    0.1    81.45781  0.5519354  58.84289
  35    0.2    78.04

In [75]:
model_list <- list(lm=lm1_cv,rf=rf1_cv,glmnet=lasso1_cv,rf=rf1_cv,nnet=nn1_cv,gbm=gbm1_cv)
res <- resamples(model_list)
# Getting the summary of the models
summary(res)


Call:
summary.resamples(object = res)

Models: lm, rf, glmnet, rf, nnet, gbm 
Number of resamples: 10 

MAE 
           Min.  1st Qu.   Median     Mean  3rd Qu.     Max. NA's
rf     31.39389 34.56492 36.73979 36.54191 37.80281 41.79440    0
lm     27.51809 34.16762 34.97423 35.92936 38.19371 45.07127    0
glmnet 29.20627 32.50154 35.77476 35.89352 37.57224 45.10500    0
nnet   46.42443 53.95139 55.33777 55.42235 57.39048 61.04822    0
gbm    33.92594 35.50666 37.90167 37.87495 39.36533 44.05849    0

RMSE 
           Min.  1st Qu.   Median     Mean  3rd Qu.     Max. NA's
rf     45.86318 51.82537 55.93055 55.77452 61.50984 64.58208    0
lm     38.05997 48.25582 49.75096 52.01668 56.46298 66.98267    0
glmnet 41.94612 46.10085 50.62107 51.99433 56.55134 64.23758    0
nnet   65.12288 73.67729 75.04247 77.75022 84.84477 93.45067    0
gbm    49.25874 50.00321 56.25624 56.20133 61.76810 64.03411    0

Rsquared 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
rf     0

In [77]:
pred_lm <- predict(lasso1_cv, data_test)
R2(data_test$PM2.5_daily,pred_lm)
MAE(data_test$PM2.5_daily,pred_lm)

[1] 0.5169162

[1] 53.38833